## 准备数据

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()

    x = x / 255.0
    x_test = x_test / 255.0
    
    return (x, y), (x_test, y_test)

## Demo numpy based auto differentiation

1. 计算两个矩阵相乘的导数

设有两个矩阵：$Y = X W$
其中：
- $ X $ 维度为 $ (N, d) $
- $ W $ 维度为 $ (d, d') $
- $ Y $ 维度为 $ (N, d') $

我们需要计算损失函数$ L $ **对 $ X $ 和$ W $ 的梯度**，即：
$$
\frac{\partial L}{\partial X}, \quad \frac{\partial L}{\partial W}
$$

假设损失 $ L $ 对 $ Y $ 的梯度为：
$$
\frac{\partial L}{\partial Y} = G \quad (N, d')
$$
我们计算：
- **对 \( X \) 的梯度：**
$$
\frac{\partial L}{\partial X} = G W^T
$$
- **对 \( W \) 的梯度：**
$$
\frac{\partial L}{\partial W} = X^T G
$$

2. ReLU函数

$$
f(x) = \max(0, x)
$$
- 当 $ x > 0 $ 时，$ f(x) = x $（保持原值）。
- 当 $ x \leq 0 $ 时，$ f(x) = 0 $（截断为 0）。

3. softmax函数

$$
s_i = \frac{e^{x_i}}{\sum_{j} e^{x_j}}
$$

对于 softmax 函数的梯度，我们需要计算对任意输入 $ x_k $ 的偏导数：

$$
\frac{\partial s_i}{\partial x_k}
$$

我们分两种情况讨论：
- **当 $ i = k $ 时（对自己求导）：**
   $$
   \frac{\partial s_i}{\partial x_i} = s_i (1 - s_i)
   $$
   
- **当 $ i \neq k $ 时（对其他项求导）：**
   $$
   \frac{\partial s_i}{\partial x_k} = -s_i s_k = s_i (0 - s_k)
   $$

将这两种情况合并，可以写成一个矩阵形式：

$$
\frac{\partial s}{\partial x} = S (I - S)
$$

其中：
- $ S $ 是 softmax 输出的 **列向量**（$ (C, 1) $ 维度）。
- $ I $ 是单位矩阵（$ (C, C) $）。
- 结果是一个 **Jacobian 矩阵**，形式如下：

$$
J_{ik} = s_i (\delta_{ik} - s_k)
$$

其中，**Kronecker delta** $\delta_{ik}$ 为：
$$
\delta_{ik} =
\begin{cases}
1, & i = k \\
0, & i \neq k
\end{cases}
$$

In [4]:
import numpy as np

class Matmul:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):
        h = np.matmul(x, W)
        self.mem={'x':x, 'W':W}
        return h
    
    def backward(self, grad_y):
        '''
        x: shape(N, d)
        w: shape(d, d')
        grad_y: shape(N, d')
        '''
        x = self.mem['x']
        W = self.mem['W']
        
        # 计算梯度
        grad_x = np.matmul(grad_y, W.T)  # 对 x 求导
        grad_W = np.matmul(x.T, grad_y)  # 对 W 求导
        
        return grad_x, grad_W


class Relu:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        grad_x = grad_y * (x > 0)  # 只有 x > 0 的位置梯度为 1
        
        return grad_x
    


class Softmax:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12 # 防止除以0
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        x_exp = np.exp(x)
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/(partition+self.epsilon)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        s = self.mem['out']
        sisj = np.matmul(np.expand_dims(s,axis=2), np.expand_dims(s, axis=1)) # (N, c, c)
        g_y_exp = np.expand_dims(grad_y, axis=1)
        tmp = np.matmul(g_y_exp, sisj) #(N, 1, c)
        tmp = np.squeeze(tmp, axis=1)
        tmp = -tmp+grad_y*s 
        return tmp
    
class Log:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        out = np.log(x+self.epsilon)
        
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        
        return 1./(x+1e-12) * grad_y
    


## Gradient check

In [12]:
# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# W = np.random.normal(size=[6, 4])
# aa = Matmul()
# out = aa.forward(x, W) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x, W = tf.constant(x), tf.constant(W)
#     tape.watch(x)
#     y = tf.matmul(x, W)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Relu()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.relu(x)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf
# x = np.random.normal(size=[5, 6], scale=5.0, loc=1)
# label = np.zeros_like(x)
# label[0, 1]=1.
# label[1, 0]=1
# label[1, 1]=1
# label[2, 3]=1
# label[3, 5]=1
# label[4, 0]=1
# print(label)
# aa = Softmax()
# out = aa.forward(x) # shape(5, 6)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.softmax(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Log()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.math.log(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# Final Gradient Check

In [13]:
import tensorflow as tf

x = np.random.normal(size=[5, 6])
W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])

label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

h1 = mul_h1.forward(x, W1) # shape(5, 4)
h1_relu = relu.forward(h1)
h2 = mul_h2.forward(h1_relu, W2)
h2_soft = softmax.forward(h2)
h2_log = log.forward(h2_soft)


h2_log_grad = log.backward(label)
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)

print(h2_log_grad)
print('--'*20)
# print(W2_grad)

with tf.GradientTape() as tape:
    x, W1, W2, label = tf.constant(x), tf.constant(W1), tf.constant(W2), tf.constant(label)
    tape.watch(W1)
    tape.watch(W2)
    h1 = tf.matmul(x, W1)
    h1_relu = tf.nn.relu(h1)
    h2 = tf.matmul(h1_relu, W2)
    prob = tf.nn.softmax(h2)
    log_prob = tf.math.log(prob)
    loss = tf.reduce_sum(label * log_prob)
    grads = tape.gradient(loss, [prob])
    print (grads[0].numpy())

[[0.00000000e+00 3.74099226e+06 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.61493133e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.94882013e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.96016977e+03]
 [8.76434451e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
----------------------------------------
[[0.00000000e+00 3.74100625e+06 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.61493133e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.94882013e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.96016977e+03]
 [8.76434451e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


## 建立模型

In [14]:
class myModel:
    def __init__(self):
        
        self.W1 = np.random.normal(size=[28*28+1, 100])
        self.W2 = np.random.normal(size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        bias = np.ones(shape=[x.shape[0], 1])
        x = np.concatenate([x, bias], axis=1)
        
        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)
        self.h1_relu = self.relu.forward(self.h1)
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)
        self.h2_soft = self.softmax.forward(self.h2)
        self.h2_log = self.log.forward(self.h2_soft)
            
    def backward(self, label):
        self.h2_log_grad = self.log.backward(-label)
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)
        self.h1_relu_grad = self.relu.backward(self.h2_grad)
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)
        
model = myModel()


## 计算 loss

In [15]:
def compute_loss(log_prob, labels):
     return np.mean(np.sum(-log_prob*labels, axis=1))
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1)
    truth = np.argmax(labels, axis=1)
    return np.mean(predictions==truth)

def train_one_step(model, x, y):
    model.forward(x)
    model.backward(y)
    model.W1 -= 1e-5* model.W1_grad
    model.W2 -= 1e-5* model.W2_grad
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

def test(model, x, y):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

## 实际训练

In [16]:
train_data, test_data = mnist_dataset()
train_label = np.zeros(shape=[train_data[0].shape[0], 10])
test_label = np.zeros(shape=[test_data[0].shape[0], 10])
train_label[np.arange(train_data[0].shape[0]), np.array(train_data[1])] = 1.
test_label[np.arange(test_data[0].shape[0]), np.array(test_data[1])] = 1.

for epoch in range(50):
    loss, accuracy = train_one_step(model, train_data[0], train_label)
    print('epoch', epoch, ': loss', loss, '; accuracy', accuracy)
loss, accuracy = test(model, test_data[0], test_label)

print('test loss', loss, '; accuracy', accuracy)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
epoch 0 : loss 23.568715096072843 ; accuracy 0.0884
epoch 1 : loss 20.946217368820303 ; accuracy 0.18731666666666666
epoch 2 : loss 19.584068618612605 ; accuracy 0.22398333333333334
epoch 3 : loss 18.661663910497342 ; accuracy 0.2633
epoch 4 : loss 17.657826336838596 ; accuracy 0.29328333333333334
epoch 5 : loss 16.925385452228763 ; accuracy 0.3193666666666667
epoch 6 : loss 16.29452983043265 ; accuracy 0.34018333333333334
epoch 7 : loss 16.050768117344603 ; accuracy 0.36201666666666665
epoch 8 : loss 14.9718748393473 ; accuracy 0.38945
epoch 9 : loss 14.263984736509775 ; accuracy 0.4177666666666667
epoch 10 : loss 14.018358734855736 ; accuracy 0.425
epoch 11 : loss 13.693648335033311 ; accuracy 0.44838333333333336
epoch 12 : loss 12.828494909880833 ; accuracy 0.47678333333333334
epoch 13 : loss 13.036121110218783 ; accuracy 0.4743
epoch 14 : loss 12.554466133637343 ; accuracy 0.49123333333333336
epoch 15 : loss 12.719340651425824 ; ac